# Installation and Setup

In [1]:
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/noisystudent/noisy_student_efficientnet-b0.tar.gz

--2021-11-17 20:29:31--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/noisystudent/noisy_student_efficientnet-b0.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 142.251.8.128, 2404:6800:4008:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156407520 (149M) [application/octet-stream]
Saving to: ‘noisy_student_efficientnet-b0.tar.gz’

noisy_student_effic 100%[===================>] 149.16M  34.7MB/s    in 4.3s    

2021-11-17 20:29:36 (34.7 MB/s) - ‘noisy_student_efficientnet-b0.tar.gz’ saved [156407520/156407520]



In [2]:
!tar -xf noisy_student_efficientnet-b0.tar.gz

In [3]:
!python efficientnet_weight_update_util.py --model b0 --notop --ckpt noisy_student_efficientnet-b0/model.ckpt --o efficientnetb0_notop.h5

2021-11-17 20:29:44.430893: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
16719872/16705208 [==============================] - 0s 0us/step
check variables match in each block
blocks_0 and block1a match.
blocks_1 and block2a match.
blocks_2 and block2b match.
blocks_3 and block3a match.
blocks_4 and block3b match.
blocks_5 and block4a match.
blocks_6 and block4b match.
blocks_7 and block4c match.
blocks_8 and block5a match.
blocks_9 and block5b match.
blocks_10 and block5c match.
blocks_11 and block6a match.
blocks_12 and block6b match.
blocks_13 and block6c match.
blocks_14 and block6d match.
blocks_15 and block7a match.
skipping variable normalization/mean:0: normalization is a layerin keras implementation, but preprocessing in TF implementation.
skipping variable normalization/variance:0: normalization is a layerin keras implementation, but p

In [4]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 4.0 MB/s 


# Data Preparation

In [11]:
import shutil
import os
import pandas as pd

# shutil.rmtree('data')

In [6]:
os.mkdir('data')

In [ ]:
!unrar x data/imgs.rar data

In [8]:
from os import listdir
from os.path import isfile, join

data_path = 'data/'
json_files = json_files = [pos_json for pos_json in listdir(data_path) if pos_json.endswith('.json')]

In [12]:
import json
df = pd.DataFrame(columns=['image_location', 'point1', 'point2'])

for json_file in json_files:
  with open(data_path+json_file) as jf:
      d = json.load(jf)
      arr = list(eval(d['eye_details']['look_vec']))
      df = df.append({'image_location': data_path+json_file.split('.')[0]+'.jpg', 'point1': arr[0], 'point2': arr[1]}, ignore_index=True)

In [13]:
df

,image_location,point1,point2
0,data/1111.jpg,0.9172,-0.0279
1,data/855.jpg,0.3415,-0.3774
2,data/91.jpg,0.7313,0.2205
3,data/325.jpg,0.2839,-0.3854
4,data/144.jpg,-0.1043,0.0695
...,...,...,...
1153,data/1086.jpg,0.0045,-0.5430
1154,data/1039.jpg,0.6823,0.0080
1155,data/731.jpg,0.2550,-0.0588
1156,data/237.jpg,0.1189,0.5486


# Main

In [14]:
# General libraries
import pandas as pd  #For working with dataframes
import numpy as np   #For working with image arrays
import cv2          #For transforming image
import matplotlib.pyplot as plt  #For representation
#For model building
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, models, utils
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from skimage import io, transform
from torch.optim import lr_scheduler
from skimage.transform import AffineTransform, warp

In [15]:
class MyData(Dataset):
    def __init__(self, df, train=True, transform=None):
        images = []
        for image in df['image_location']:
          img = Image.open(image)
          type(img)
          images.append(img)
        #Setting labels
        label_point1=df['point1']
        label_point2=df['point2']
        
        #Splitting the data into train and validation set
        X_train, X_test, y_train_point1, y_test_point1,  y_train_point2,\
        y_test_point2 = train_test_split(images, label_point1, label_point2, test_size=0.2)
        
        if train==True: 
            self.x=X_train
            self.y_point1=y_train_point1
            self.y_point2=y_train_point2
        else:
            self.x=X_test
            self.y_point1=y_test_point1
            self.y_point2=y_test_point2            
        
        #Applying transformation
        self.transform=transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        image=self.x[idx]
        label1=np.array([self.y_point1.iloc[idx]]).astype('float')
        label2=np.array([self.y_point2.iloc[idx]]).astype('float')
        
        sample={'image': image, 'label_point1': label1,\
                'label_point2': label2}
        
        #Applying transformation
        if self.transform:
            sample={'image': self.transform(image), 'label_point1': label1,\
                'label_point2': label2}
            
        return sample

In [124]:
a = MyData(df, train=True, transform=transforms)
a.__getitem__(2)

{'image': tensor([[[0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          ...,
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804]],
 
         [[0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          ...,
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804]],
 
         [[0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.3804,  ..., 0.3804, 0.3804, 0.3804],
          [0.3804, 0.3804, 0.38

In [16]:
transforms = T.Compose([
    T.Resize((128, 128)),
    T.ToTensor()
])

In [17]:
train_data = MyData(df, train=True, transform=transforms)
test_data = MyData(df, train=False, transform=transforms)

train_dataloader = DataLoader(train_data, batch_size=50, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=50, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [24]:
class CNN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        self.fc1 = nn.Linear(1280, 1)  #For age class
        self.fc2 = nn.Linear(1280, 1)    #For gender class
        
    def forward(self, x):
        bs, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        label1 = self.fc1(x)
        label2 = self.fc2(x)
        return {'label1': label1, 'label2': label2}

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
#Setting model and moving to device
model_CNN = CNN1().to(device)
#For binary output:gender
loss = nn.MSELoss()
optimizer = optim.SGD(model_CNN.parameters(), lr=0.001, momentum=0.9)

In [27]:
def train_model(model, loss_func, optimizer, n_epochs=25):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    for epoch in range(1, n_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        # train the model #
        model.train()
        for batch_idx, sample_batched in enumerate(train_dataloader):
            # importing data and moving to GPU
            image, label1, label2 = sample_batched['image'].to(device),\
                                             sample_batched['label_point1'].to(device),\
                                              sample_batched['label_point2'].to(device)  
            # zero the parameter gradients
            optimizer.zero_grad()
            output=model(image)
            label1_hat=output['label1']
            label2_hat=output['label2']        
            # calculate loss
            loss1=loss_func(label1_hat, label1.squeeze().float())
            loss2=loss_func(label2_hat, label2.squeeze().float())   
            loss=loss1+loss2
            # back prop
            loss.backward()
            # grad
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            if batch_idx % 50 == 0:
                print('Epoch %d, Batch %d loss: %.6f' %
                  (epoch, batch_idx + 1, train_loss))
        # validate the model #
        model.eval()
        for batch_idx, sample_batched in enumerate(test_dataloader):
            image, label1, label2 = sample_batched['image'].to(device),\
                                             sample_batched['label_point1'].to(device),\
                                              sample_batched['label_point2'].to(device) 
            output = model(image)
            label1_hat=output['label1']
            label2_hat=output['label2']               
            # calculate loss
            loss1=loss_func(label1_hat, label1.squeeze())
            loss2=loss_func(label2_hat, label2.squeeze())
            loss=loss1+loss2
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model, 'model.pt')
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
    # return trained model
    return model

In [28]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [29]:
model_conv=train_model(model_CNN, loss, optimizer)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Batch 1 loss: 0.468952


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([26])) that is different to the input size (torch.Size([26, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1 	Training Loss: 0.400282 	Validation Loss: 0.401379
Validation loss decreased (inf --> 0.401379).  Saving model ...


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2, Batch 1 loss: 0.350377
Epoch: 2 	Training Loss: 0.366208 	Validation Loss: 0.363178
Validation loss decreased (0.401379 --> 0.363178).  Saving model ...
Epoch 3, Batch 1 loss: 0.371155
Epoch: 3 	Training Loss: 0.349809 	Validation Loss: 0.361622
Validation loss decreased (0.363178 --> 0.361622).  Saving model ...
Epoch 4, Batch 1 loss: 0.368379
Epoch: 4 	Training Loss: 0.341572 	Validation Loss: 0.345551
Validation loss decreased (0.361622 --> 0.345551).  Saving model ...
Epoch 5, Batch 1 loss: 0.321118
Epoch: 5 	Training Loss: 0.335691 	Validation Loss: 0.344003
Validation loss decreased (0.345551 --> 0.344003).  Saving model ...
Epoch 6, Batch 1 loss: 0.347479
Epoch: 6 	Training Loss: 0.331735 	Validation Loss: 0.348494
Epoch 7, Batch 1 loss: 0.309237
Epoch: 7 	Training Loss: 0.328854 	Validation Loss: 0.330960
Validation loss decreased (0.344003 --> 0.330960).  Saving model ...
Epoch 8, Batch 1 loss: 0.336635
Epoch: 8 	Training Loss: 0.333984 	Validation Loss: 0.335334
Epoc

In [30]:
import cv2
from google.colab.patches import cv2_imshow

def draw(model):
  for batch_idx, sample_batched in enumerate(train_dataloader):
    # importing data and moving to GPU
    image, label1, label2 = sample_batched['image'].to(device),\
                                      sample_batched['label_point1'].to(device),\
                                      sample_batched['label_point2'].to(device)  
    # zero the parameter gradients
    optimizer.zero_grad()
    output=model(image)
    x=output['label1']
    y=output['label2'] 
    # print(type(x.cpu()))
    print(image[0].shape)
    cv2.circle(image[0].permute(1, 2, 0), (x, y), 1, (0, 255, 0), 1)

    cv2_imshow(image[0])
    break

In [31]:
draw(model_conv)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([3, 128, 128])


TypeError: ignored